In [14]:
# Image stitching
# Note: install opencv-contrib-python
import cv2
import numpy as np
import matplotlib.pyplot as plt

resize_height = 400
resize_width = 400

ratio_val = 0.75
threshold_val = 4

def read_image(an_image):
    """This function reads any image from a given path"""
    image = cv2.imread(an_image)
    return image

def resize_image(an_image,r_w, r_h):
    """function to resize an original image to a specified dimension giving the resize height and resize width"""
    resized_image = cv2.resize(an_image,(r_w,r_h),interpolation=cv2.INTER_AREA)
    return resized_image


def gray_image(an_image):
    """function to convert BGR image to Gray"""
    return cv2.cvtColor(an_image, cv2.COLOR_BGR2GRAY)


def keypoint_and_features_function(grayscale_resized_image):
    # create an instance of the object detector and descriptor
    descriptor = cv2.xfeatures2d.SIFT_create()
    (keypoints,features) = descriptor.detectAndCompute(grayscale_resized_image,None)
    
    Key_points = []
    
    for pts in keypoints:
        pt = np.float32(pts.pt)
        Key_points.append(pt)
        
    return (Key_points,features)


def match_points(first_keypoint_image, second_keypoint_image,first_features, second_features, ratio, thresh):
    match_mthd = cv2.DescriptorMatcher_create("BruteForce")
    
    # using K-Nearest Neighbor method to match the features
    # setting the number of nearest neighbors to 2
    nearest_neighbor = 2
    raw_matches = match_mthd.knnMatch(first_features,second_features,nearest_neighbor)
    
    matches = []
    
    # removing any false positives
    for match in raw_matches:
        # Checking the distance of the two matches with the ratio under consideration
        if len(match) == 2 and match[0].distance < match[1].distance * ratio:
            matches.append((match[0].trainIdx, match[0].queryIdx))
    
    # A minimum of four (4) matches are required to calculate the homography matrix
    # The matrix contains the information to warp the images
    
    if len(matches) > 4:
        points_image1 = []
        points_image2 = []
        
        for _, index in matches:
            pts = np.float32(first_keypoint_image[index])
            pts = first_keypoint_image[index]
            points_image1.append(pts)
        
        for index,_ in matches:
            pts = np.float32(second_keypoint_image[index])
            pts = second_keypoint_image[index]
            points_image2.append(pts)
        
        points_image1 = np.array(points_image1, dtype=np.float32)
        points_image2 = np.array(points_image2, dtype=np.float32)
        
        (Homography_matrix, status) = cv2.findHomography(points_image1, points_image2, cv2.RANSAC,thresh)
        
        return (matches, Homography_matrix, status)
    
    return None
    

def display_image(window_name, specified_image):
    """This function display any image from a given path"""
    cv2.imshow(window_name, specified_image)
    cv2.waitKey(0)


def matching_imagepoints_comp(an_image1,an_image2,a_keypoint_image1,a_keypoint_image2):
    result = cv2.drawMatchesKnn(an_image1,a_keypoint_image1,an_image2,a_keypoint_image2,matches,status, flags=0)
    
    return result
    

    

    
# Begin of code run

# Read first and second image using the read image function
first_image = read_image(an_image='image1.png')
second_image = read_image(an_image='image2.png')

# Resize first and second image using the resize image function
resized_first_image = resize_image(an_image=first_image,r_w=resize_width, r_h=resize_height)
resized_second_image = resize_image(an_image=second_image,r_w=resize_width, r_h=resize_height)

# convert first and second image to gray scale using the gray scale image conversion function
gray_resized_first_image = gray_image(an_image=resized_first_image)
gray_resized_second_image = gray_image(an_image=resized_second_image)

#Extracting the keypoints and features from first and second image using the keypoints feature function
(keypoints_image1, features_image1) = keypoint_and_features_function(grayscale_resized_image=gray_resized_second_image)
(keypoints_image2, features_image2) = keypoint_and_features_function(grayscale_resized_image=gray_resized_first_image)

# Finding the match points in both images
M = match_points(first_keypoint_image=keypoints_image1, second_keypoint_image=keypoints_image2,first_features=features_image1, 
                 second_features=features_image2,ratio=ratio_val, thresh=threshold_val)

if M is None:
    print("No Matches detected")
    exit()

matches, Homography_matrix, status = M

stitched_image = cv2.warpPerspective(second_image,Homography_matrix, (second_image.shape[1] + first_image.shape[1],first_image.shape[0]))
stitched_image[0:first_image.shape[0], 0:first_image.shape[1]] = first_image

# show the points in first image that matches with points in second image
keypoints_matching_image = matching_imagepoints_comp(an_image1=first_image,an_image2=second_image,a_keypoint_image1=keypoints_image1,
                                                     a_keypoint_image2=keypoints_image2)


display_image(window_name='Keypoints Matches', specified_image=keypoints_matching_image)
display_image(window_name='Stitched Images', specified_image=stitched_image)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'drawMatchesKnn'
> Overload resolution failed:
>  - Can't parse 'keypoints1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'keypoints1'. Sequence item with index 0 has a wrong type
